In [6]:
# Importing libraries 
import os
import sys
import glob
import numpy as np
import scipy.optimize as op
import matplotlib.pyplot as plt
import urllib.request
from astropy.io import fits
from astropy.table import Table
from astropy.table import vstack
from astropy.io import fits
import urllib.request
import requests
import time

In [8]:

#Getting the individual coadded files names

local_dir = "/home/colgoat/spc_DESIedr/" #path to the catalogs
pn_sv1 = "https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/healpix/sv1/dark/" # File database link for sv1
pn_sv3 = "https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/healpix/sv3/dark/" # File database link for sv3

EDR_qsolist = [] # List of spectra of interest from DESI EDR

print("\n")
print("---------------------------------------------")
print("\n")

file1 = "QSO_cat_fuji_sv1_dark_bal_mask_v1.0.fits" # Name of the first catalog (sv1)
lc_sv1 = os.path.join(local_dir,file1) # lc_sv1 = Local file sv1
fc_sv1 = fits.open(lc_sv1) # fc_sv1 = Open .fits catalog file of sv1
SV1 = Table.read(fc_sv1) # SV1 = Reading the file of SV1 as a BinTable 
q_sv1 = (SV1['Z']>=2.1) & (SV1['PROGRAM']=='dark') # q_sv1 = Query from main catalog (filtering)
catfil_sv1 = SV1[q_sv1] # catfil_sv1 = Filtered catalog sv1
catfil_sv1_hpix = catfil_sv1.group_by("HPXPIXEL") # catfil_sv1 = Grouping filtered catalog by HEALPIX criteria
HPX_sv1 = catfil_sv1_hpix["HPXPIXEL"]/100 # HPX_sv1 = Getting the healpix ID for path on database

print("Number of spectra in SV1: ",len(catfil_sv1)) # Showing the number of targets on screen
for p,m in zip(HPX_sv1,catfil_sv1_hpix["HPXPIXEL"]): # Adding selected Targets to the list
    EDR_qsolist.append(pn_sv1+str(int(p))+"/"+str(m)+"/coadd-sv1-dark-"+str(m)+".fits")
fc_sv1.close()

file2 = "QSO_cat_fuji_sv3_dark_bal_mask_v1.0.fits" # Name of the first catalog (sv3)
lc_sv3 = os.path.join(local_dir,file2) # lc_sv3 = Local file sv3
fc_sv3 = fits.open(lc_sv3) # fc_sv3 = Open .fits catalog file of sv3
SV3 = Table.read(fc_sv3) # SV3 = Reading the file of SV3 as a BinTable
q_sv3 = (SV3['Z']>=2.1) & (SV3['PROGRAM']=='dark') # q_sv3 = Query from main catalog (filtering)
catfil_sv3 = SV3[q_sv3] # catfil_sv3 = Filtered catalog sv3
catfil_sv3_hpix = catfil_sv3.group_by("HPXPIXEL") # catfil_sv3 = Grouping filtered catalog by HEALPIX criteria
HPX_sv3 = catfil_sv3_hpix["HPXPIXEL"]/100 # HPX_sv3 = Getting the healpix ID for path on database

print("Number of spectra in SV3: ",len(catfil_sv3))
for p,m in zip(HPX_sv3,catfil_sv3_hpix["HPXPIXEL"]): # Adding selected Targets to the list
    EDR_qsolist.append(pn_sv3+str(int(p))+"/"+str(m)+"/coadd-sv3-dark-"+str(m)+".fits")
fc_sv3.close()


EDRcat_stk = vstack(([catfil_sv1,catfil_sv3])) # EDR filtered catalogs stacked for both surveys and sorted
EDRcat_stk.sort(['TARGETID'])


print("Total number of DESI EDR spectra (SV1 & SV3) to be used: ",len(EDR_qsolist))




---------------------------------------------


Number of spectra in SV1:  11853
Number of spectra in SV3:  14910
Total number of DESI EDR spectra (SV1 & SV3) to be used:  26763


In [59]:
cor_cat = Table.read(fits.open('/home/colgoat/spc_DESIedr/qso_catalog_sv13.fits')[1])
#cor_cat[(cor_cat['PROGRAM']=='dark') & ((cor_cat['SURVEY']=='sv1') | (cor_cat['SURVEY']=='sv3'))]
cor_cat.sort(['TARGETID'])

In [71]:
C1_TID = np.array(EDRcat_stk['TARGETID'])
C2_TID = np.array(cor_cat['TARGETID'])
A = np.intersect1d(C2_TID,C1_TID)
print(len(C1_TID), len(C2_TID), len(A))
mask = np.isin(C1_TID,(np.intersect1d(C2_TID,C1_TID)))
B = EDRcat_stk[mask]

26763 68750 21133
21133


In [78]:
dup = []
for a in range(len(A)):
    ix = (C1_TID == A[a]) 
    dup.append(ix.sum())
dup = np.array(dup)
np.unique(dup)
len(dup)

21133

In [ ]:
mask = np.isin(C2_TID,C1_TID)
len(mask)

In [41]:
new_CORcat = cor_cat[mask]
new_CORcat.sort(['TARGETID'])

In [42]:
C1_TID = np.array(new_CORcat['TARGETID'])
C2_TID = np.array(EDRcat_stk['TARGETID'])
mask = np.isin(C2_TID,C1_TID, assume_unique = True)
len(mask)

26763

In [43]:
new_BALcat = EDRcat_stk[mask]
new_BALcat.sort(['TARGETID'])

21133

In [34]:
new_CORcat

TARGETID,Z,ZERR,ZWARN,SPECTYPE,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,PROBA_RF,MASKBITS,SV1_DESI_TARGET,SV1_SCND_TARGET,DESI_TARGET,COADD_NUMEXP,COADD_EXPTIME,CMX_TARGET,SV2_DESI_TARGET,SV3_DESI_TARGET,SV2_SCND_TARGET,SV3_SCND_TARGET,SCND_TARGET,TSNR2_LYA,TSNR2_QSO,DELTA_CHI2_MGII,A_MGII,SIGMA_MGII,B_MGII,VAR_A_MGII,VAR_SIGMA_MGII,VAR_B_MGII,Z_RR,Z_QN,C_LYA,C_CIV,C_CIII,C_MgII,C_Hbeta,C_Halpha,Z_LYA,Z_CIV,Z_CIII,Z_MgII,Z_Hbeta,Z_Halpha,QSO_MASKBITS,HPXPIXEL,SURVEY,PROGRAM,LASTNIGHT,PCA_COEFFS,PCA_CHI2,BAL_PROB,BI_CIV,ERR_BI_CIV,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,POSMIN_CIV_2000,FMIN_CIV_2000,AI_CIV,ERR_AI_CIV,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,POSMIN_CIV_450,FMIN_CIV_450,BI_SIIV,ERR_BI_SIIV,NSIIV_2000,VMIN_SIIV_2000,VMAX_SIIV_2000,POSMIN_SIIV_2000,FMIN_SIIV_2000,AI_SIIV,ERR_AI_SIIV,NSIIV_450,VMIN_SIIV_450,VMAX_SIIV_450,POSMIN_SIIV_450,FMIN_SIIV_450,BALMASK
int64,float64,float64,int64,str6,int32,float64,float64,str4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,int16,int64,int64,int64,int16,float32,int64,int64,int64,int64,int64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int64,str3,str4,int64,float32[5],float32,float32,float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],uint8
39627136847319542,2.334165306571836,0.00011927768655614493,0,QSO,0,163.32487960346054,-27.909257960373424,PSF,0.06449058,8.060436,9.266329,13.961395,17.785593,32.406742,215.88075,199.37141,63.222946,1.7885725,0.5398294,0.8058177000368452,0.8058177000368452,0.8058177000368452,0.8058177000368452,0.8058177000368452,0.9981405733823776,0,4665733337124491014,131072,0,19,17061.107,0,0,0,0,0,0,270.09314,130.79248,361.8064,0.602876,44.95604,2.3683345,0.0016881061,17.090488,0.0011731663,2.3341653,2.337243,0.9999999,0.9999998,0.99999934,0.9999992,2.1405694e-05,1.9597617e-05,2.337243,2.3374805,2.3341882,2.333792,-0.013425438,0.46398482,14,38771,sv1,dark,20210212,229.6426 .. 3.6772516,2.3033245,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,258.45255,2.6098938,2,5701.53 .. -1.0,6444.535 .. -1.0,5887.2812 .. -1.0,0.7411206 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0
39627142190859939,2.175128666031821,0.0002438795433343139,0,QSO,0,162.88450856962623,-27.774641891054515,PSF,0.06621869,0.48587924,0.4227278,0.7662796,3.0602136,2.5776358,544.62964,481.0964,190.13463,3.5453265,0.7686863,0.8011692907279641,0.8011692907279641,0.8011692907279641,0.8011692907279641,0.8011692907279641,0.9618029569089412,0,72061717206601478,0,0,19,17061.107,0,0,0,0,0,0,292.42,141.81735,82.94344,0.089525715,-22.230644,0.09703628,0.00076991005,78.62632,0.0001402657,2.1751287,2.1828747,0.9999669,1.0,0.99999774,0.99996704,4.0566556e-06,4.4175246e-07,2.1860728,2.1828747,2.1892407,2.1893008,-0.06362737,-0.45145366,14,38771,sv1,dark,20210212,11.221055 .. 0.9191993,1.4971107,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0
39627152919890882,2.453393365329811,0.00045964276141543354,0,QSO,0,162.7418127945335,-27.319436661954988,PSF,0.074578695,0.35029408,0.37503088,0.69750464,3.7208774,3.2596428,732.7517,720.92444,183.09274,3.4404063,0.742372

In [8]:
cor_cat = Table.read(fits.open('/home/colgoat/spc_DESIedr/zall-pix-fuji.fits')[1])
cor_cat[(cor_cat['PROGRAM']=='dark') & ((cor_cat['SURVEY']=='sv1') | (cor_cat['SURVEY']=='sv3'))]

TARGETID,SURVEY,PROGRAM,HEALPIX,SPGRPVAL,Z,ZERR,ZWARN,CHI2,COEFF,NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,REF_EPOCH,FA_TARGET,FA_TYPE,OBJTYPE,SUBPRIORITY,OBSCONDITIONS,RELEASE,BRICKNAME,BRICKID,BRICK_OBJID,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,MASKBITS,SERSIC,SHAPE_R,SHAPE_E1,SHAPE_E2,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_RP_MEAN_MAG,PARALLAX,PHOTSYS,PRIORITY_INIT,NUMOBS_INIT,CMX_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,SCND_TARGET,SV1_DESI_TARGET,SV1_BGS_TARGET,SV1_MWS_TARGET,SV1_SCND_TARGET,SV2_DESI_TARGET,SV2_BGS_TARGET,SV2_MWS_TARGET,SV2_SCND_TARGET,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SV3_SCND_TARGET,PLATE_RA,PLATE_DEC,COADD_NUMEXP,COADD_EXPTIME,COADD_NUMNIGHT,COADD_NUMTILE,MEAN_DELTA_X,RMS_DELTA_X,MEAN_DELTA_Y,RMS_DELTA_Y,MEAN_FIBER_RA,STD_FIBER_RA,MEAN_FIBER_DEC,STD_FIBER_DEC,MEAN_PSF_TO_FIBER_SPECFLUX,TSNR2_GPBDARK_B,TSNR2_ELG_B,TSNR2_GPBBRIGHT_B,TSNR2_LYA_B,TSNR2_BGS_B,TSNR2_GPBBACKUP_B,TSNR2_QSO_B,TSNR2_LRG_B,TSNR2_GPBDARK_R,TSNR2_ELG_R,TSNR2_GPBBRIGHT_R,TSNR2_LYA_R,TSNR2_BGS_R,TSNR2_GPBBACKUP_R,TSNR2_QSO_R,TSNR2_LRG_R,TSNR2_GPBDARK_Z,TSNR2_ELG_Z,TSNR2_GPBBRIGHT_Z,TSNR2_LYA_Z,TSNR2_BGS_Z,TSNR2_GPBBACKUP_Z,TSNR2_QSO_Z,TSNR2_LRG_Z,TSNR2_GPBDARK,TSNR2_ELG,TSNR2_GPBBRIGHT,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBBACKUP,TSNR2_QSO,TSNR2_LRG,SV_NSPEC,SV_PRIMARY,ZCAT_NSPEC,ZCAT_PRIMARY
int64,str7,str6,int32,int32,float64,float64,int64,float64,float64[10],int64,str6,str20,int64,float64,int32,float64,float64,float32,float32,float32,int64,uint8,str3,float64,int32,int16,str8,int32,int32,str4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,int64,str2,float32,float32,float32,float32,str1,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int16,float32,int16,int16,float32,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,bool,int64,bool
39628203584653060,sv1,dark,4678,4678,-0.0019956912923479522,4.1311493573349107e-48,1570,8.999999999999996e+99,0.0 .. 0.0,0,STAR,CV,3,1.942668892225729e+84,512,130.10870428253665,17.554899097969106,0.0,0.0,2021.1047,824633721602,1,TGT,0.9305772542840636,3,9010,1301p175,430231,1796,EXP,0.024541996,0.6834641,1.3740861,2.631381,1.99353,6.2011814,910.74146,162.94249,48.262375,2.7454386,0.58532184,0.31644726,0.63620865,1.2183423,0.31893387,0.64112645,1.2244456,0,1.0,0.85883695,-0.30267748,-0.41440308,0,--,0.0,0.0,0.0,0.0,S,3000,1,0,0,0,0,0,824633721602,0,0,0,0,0,0,0,0,0,0,0,130.10870428253665,17.554899097969106,0,0.0,0,0,-2.1088333,2.108845,2.036,2.0360227,130.10031224526466,0.112740636,17.54755228489917,0.12959233,0.0,1311.8525,0.56829333,1225.0863,171.29367,16514.455,962.2802,7.559137,19.663568,75005.06,174.29616,66089.29,0.120630175,73447.51,44435.41,25.931755,747.1752,0.00015246919,588.0757,0.00013955399,0.0,134628.9,0.00010374343,62.928356,850.02905,76316.914,762.9401,67314.375,171.4143,224590.88,45397.69,96.41925,1616.8678,1,True,1,True
616088955888075517,sv1,dark,4678,4678,0.44367792773914005,7.584055280579969e-05,1,9649.336306087673,-30.64912006769693 .. -5.227707389715611,7928,GALAXY,--,10,12.71258968859911,0,130.05856537833105,17.531232321212844,0.0,0.0,0.0,4294967296,4,SKY,0.9330034619506492,63,9010,1301p175,430231,765,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0043316763,-0.0051796963,0.03235035,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,--,0.0,0.0,0.0,0.0,--,-1,-1,0,4294967296,0,0,0,0,0,0,0,0,0,0

In [94]:
from astropy.io import fits

archivo = fits.open('/home/colgoat/spc_DESIedr/DESI_EDRcat_BM_1.0(sv13).fits')
hdu = fits.BinTableHDU(EDRcat_stk)
archivo[1] = hdu
archivo.close()

with fits.open('/home/colgoat/spc_DESIedr/DESI_EDRcat_BM_1.0(sv13).fits', mode='update') as passer:
    passer[1] = hdu
    hdr = passer[1].header
    hdr['EXTNAME'] = 'CATALOG'
    passer.flush()  # changes are written back to original.fits

In [95]:
pruf=Table.read('/home/colgoat/spc_DESIedr/DESI_EDRcat_BM_1.0(sv13).fits')
pruf

TARGETID,Z,ZERR,ZWARN,SPECTYPE,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,PROBA_RF,MASKBITS,SV1_DESI_TARGET,SV1_SCND_TARGET,DESI_TARGET,COADD_NUMEXP,COADD_EXPTIME,CMX_TARGET,SV2_DESI_TARGET,SV3_DESI_TARGET,SV2_SCND_TARGET,SV3_SCND_TARGET,SCND_TARGET,TSNR2_LYA,TSNR2_QSO,DELTA_CHI2_MGII,A_MGII,SIGMA_MGII,B_MGII,VAR_A_MGII,VAR_SIGMA_MGII,VAR_B_MGII,Z_RR,Z_QN,C_LYA,C_CIV,C_CIII,C_MgII,C_Hbeta,C_Halpha,Z_LYA,Z_CIV,Z_CIII,Z_MgII,Z_Hbeta,Z_Halpha,QSO_MASKBITS,HPXPIXEL,SURVEY,PROGRAM,PCA_COEFFS,PCA_CHI2,BAL_PROB,BI_CIV,ERR_BI_CIV,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,POSMIN_CIV_2000,FMIN_CIV_2000,AI_CIV,ERR_AI_CIV,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,POSMIN_CIV_450,FMIN_CIV_450,BI_SIIV,ERR_BI_SIIV,NSIIV_2000,VMIN_SIIV_2000,VMAX_SIIV_2000,POSMIN_SIIV_2000,FMIN_SIIV_2000,AI_SIIV,ERR_AI_SIIV,NSIIV_450,VMIN_SIIV_450,VMAX_SIIV_450,POSMIN_SIIV_450,FMIN_SIIV_450,BALMASK,Z_MASK,ZERR_MASK,ZWARN_MASK,CHI2_MASK,DELTACHI2_MASK,SPECTYPE_MASK,SNR_CIV
,,,,,,deg,deg,,mag,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,,,,,,,,
int64,float64,float64,int64,bytes6,int32,float64,float64,bytes4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,int16,int64,int64,int64,int16,float32,int64,int64,int64,int64,int64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int64,bytes3,bytes4,float32[5],float32,float32,float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],uint8,float64,float64,int64,float64,float64,bytes6,float64
39628215118992442,2.1837342473623393,0.00013626615461198534,0,QSO,0,130.64931448777944,17.92248565776863,PSF,0.023616284,3.5690012,3.1463985,4.338265,7.2705436,6.3402815,1239.8733,408.09125,75.880165,2.8767319,0.6142675,0.9239833775496703,0.9239833775496703,0.9239833775496703,0.9239833775496703,0.9239833775496703,0.9943965839147568,0,54047318697103110,0,0,6,6083.957,0,0,0,0,0,0,293.5955,112.386826,23.017323,0.21148743,-35.7064,0.68574977,0.0004154714,21.983559,0.0001771788,2.1837342,2.181272,1.0,0.8930895,0.9996561,0.9999963,2.249572e-05,1.7975288e-05,2.181272,2.1808524,2.188304,2.1916366,-0.19505365,-0.1221109,14,4679,sv1,dark,61.846046 .. 2.4299164,1.4206071,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0,-1.0,-1.0,-1,-1.0,-1.0,-1,4.686259937194885
39628209360211028,2.379998216697697,0.0001367309022860157,0,QSO,0,130.64326258132235,17.790880437347397,PSF,0.02099619,5.5868263,6.355662,8.894712,17.68829,29.616795,472.5694,231.43127,51.24521,2.6626232,0.5704455,0.9321236482870953,0.9321236482870953,0.9321236482870953,0.9321236482870953,0.9321236482870953,0.9981658909320831,0,4665733337124491014,34359869440,0,6,6083.957,0,0,0,0,0,0,299.52124,115.79789,252.642,0.8162872,27.105427,1.2413735,0.0011176108,1.7980521,0.0002412979,2.3799982,2.381573,0.9999887,0.99999917,0.99999946,0.9999993,3.1276253e-05,5.6267647e-05,2.3844068,2.3840988,2.381573,2.380935,0.3083418,-0.20571752,14,4679,sv1,dark,106.65552 .. -2.613535,1.7681665,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0

In [96]:
BALS = fits.open('/home/colgoat/spc_DESIedr/DESI_EDRcat_BM_1.0(sv13).fits')
BALS.info()

Filename: /home/colgoat/spc_DESIedr/DESI_EDRcat_BM_1.0(sv13).fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  CATALOG       1 BinTableHDU    289   26763R x 104C   [K, D, D, K, 6A, J, D, D, 4A, E, E, E, E, E, E, E, E, E, E, E, D, D, D, D, D, D, I, K, K, K, I, E, K, K, K, K, K, K, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, J, K, 3A, 4A, 5E, E, E, E, E, J, 5E, 5E, 5E, 5E, E, E, J, 17E, 17E, 17E, 17E, E, E, J, 5E, 5E, 5E, 5E, E, E, J, 17E, 17E, 17E, 17E, B, D, D, K, D, D, 6A, D]   


In [97]:
ZCAT = fits.open('/home/colgoat/spc_DESIedr/qso_catalog_sv13.fits')
ZCAT.info()

Filename: /home/colgoat/spc_DESIedr/qso_catalog_sv13.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       6   ()      
  1  ZCATALOG      1 BinTableHDU    205   68750R x 98C   [K, D, D, K, 6A, J, D, D, 4A, E, E, E, E, E, E, E, E, E, E, E, D, D, D, D, D, D, I, K, K, K, I, E, K, K, K, K, K, K, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, J, K, 3A, 4A, K, 5E, E, E, E, E, J, 5E, 5E, 5E, 5E, E, E, J, 17E, 17E, 17E, 17E, E, E, J, 5E, 5E, 5E, 5E, E, E, J, 17E, 17E, 17E, 17E, B]   


In [98]:
import astropy.units as u
from astropy.coordinates import SkyCoord

In [100]:
A1 = Table.read(fits.open('/home/colgoat/spc_DESIedr/DESI_EDRcat_BM_1.0(sv13).fits')[1])
A2 = Table.read(fits.open('/home/colgoat/spc_DESIedr/qso_catalog_sv13.fits')[1])

In [126]:
tar_balcat = np.array(A1['TARGETID'])
match_corcat = np.isin(A2["TARGETID"],tar_balcat)
chosen_ibool = np.invert(match_corcat)
idx = np.where(chosen_ibool)

In [133]:
B = A2[match_corcat]
B = B.group_by("TARGETID")
B

TARGETID,Z,ZERR,ZWARN,SPECTYPE,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,PROBA_RF,MASKBITS,SV1_DESI_TARGET,SV1_SCND_TARGET,DESI_TARGET,COADD_NUMEXP,COADD_EXPTIME,CMX_TARGET,SV2_DESI_TARGET,SV3_DESI_TARGET,SV2_SCND_TARGET,SV3_SCND_TARGET,SCND_TARGET,TSNR2_LYA,TSNR2_QSO,DELTA_CHI2_MGII,A_MGII,SIGMA_MGII,B_MGII,VAR_A_MGII,VAR_SIGMA_MGII,VAR_B_MGII,Z_RR,Z_QN,C_LYA,C_CIV,C_CIII,C_MgII,C_Hbeta,C_Halpha,Z_LYA,Z_CIV,Z_CIII,Z_MgII,Z_Hbeta,Z_Halpha,QSO_MASKBITS,HPXPIXEL,SURVEY,PROGRAM,LASTNIGHT,PCA_COEFFS,PCA_CHI2,BAL_PROB,BI_CIV,ERR_BI_CIV,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,POSMIN_CIV_2000,FMIN_CIV_2000,AI_CIV,ERR_AI_CIV,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,POSMIN_CIV_450,FMIN_CIV_450,BI_SIIV,ERR_BI_SIIV,NSIIV_2000,VMIN_SIIV_2000,VMAX_SIIV_2000,POSMIN_SIIV_2000,FMIN_SIIV_2000,AI_SIIV,ERR_AI_SIIV,NSIIV_450,VMIN_SIIV_450,VMAX_SIIV_450,POSMIN_SIIV_450,FMIN_SIIV_450,BALMASK
int64,float64,float64,int64,str6,int32,float64,float64,str4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,int16,int64,int64,int64,int16,float32,int64,int64,int64,int64,int64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int64,str3,str4,int64,float32[5],float32,float32,float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],uint8
39627136847319542,2.334165306571836,0.00011927768655614493,0,QSO,0,163.32487960346054,-27.909257960373424,PSF,0.06449058,8.060436,9.266329,13.961395,17.785593,32.406742,215.88075,199.37141,63.222946,1.7885725,0.5398294,0.8058177000368452,0.8058177000368452,0.8058177000368452,0.8058177000368452,0.8058177000368452,0.9981405733823776,0,4665733337124491014,131072,0,19,17061.107,0,0,0,0,0,0,270.09314,130.79248,361.8064,0.602876,44.95604,2.3683345,0.0016881061,17.090488,0.0011731663,2.3341653,2.337243,0.9999999,0.9999998,0.99999934,0.9999992,2.1405694e-05,1.9597617e-05,2.337243,2.3374805,2.3341882,2.333792,-0.013425438,0.46398482,14,38771,sv1,dark,20210212,229.6426 .. 3.6772516,2.3033245,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,258.45255,2.6098938,2,5701.53 .. -1.0,6444.535 .. -1.0,5887.2812 .. -1.0,0.7411206 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0
39627142190859939,2.175128666031821,0.0002438795433343139,0,QSO,0,162.88450856962623,-27.774641891054515,PSF,0.06621869,0.48587924,0.4227278,0.7662796,3.0602136,2.5776358,544.62964,481.0964,190.13463,3.5453265,0.7686863,0.8011692907279641,0.8011692907279641,0.8011692907279641,0.8011692907279641,0.8011692907279641,0.9618029569089412,0,72061717206601478,0,0,19,17061.107,0,0,0,0,0,0,292.42,141.81735,82.94344,0.089525715,-22.230644,0.09703628,0.00076991005,78.62632,0.0001402657,2.1751287,2.1828747,0.9999669,1.0,0.99999774,0.99996704,4.0566556e-06,4.4175246e-07,2.1860728,2.1828747,2.1892407,2.1893008,-0.06362737,-0.45145366,14,38771,sv1,dark,20210212,11.221055 .. 0.9191993,1.4971107,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0
39627152919890882,2.453393365329811,0.00045964276141543354,0,QSO,0,162.7418127945335,-27.319436661954988,PSF,0.074578695,0.35029408,0.37503088,0.69750464,3.7208774,3.2596428,732.7517,720.92444,183.09274,3.4404063,0.742372

In [134]:
A1 = A1.group_by("TARGETID")
A1

TARGETID,Z,ZERR,ZWARN,SPECTYPE,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,PROBA_RF,MASKBITS,SV1_DESI_TARGET,SV1_SCND_TARGET,DESI_TARGET,COADD_NUMEXP,COADD_EXPTIME,CMX_TARGET,SV2_DESI_TARGET,SV3_DESI_TARGET,SV2_SCND_TARGET,SV3_SCND_TARGET,SCND_TARGET,TSNR2_LYA,TSNR2_QSO,DELTA_CHI2_MGII,A_MGII,SIGMA_MGII,B_MGII,VAR_A_MGII,VAR_SIGMA_MGII,VAR_B_MGII,Z_RR,Z_QN,C_LYA,C_CIV,C_CIII,C_MgII,C_Hbeta,C_Halpha,Z_LYA,Z_CIV,Z_CIII,Z_MgII,Z_Hbeta,Z_Halpha,QSO_MASKBITS,HPXPIXEL,SURVEY,PROGRAM,PCA_COEFFS,PCA_CHI2,BAL_PROB,BI_CIV,ERR_BI_CIV,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,POSMIN_CIV_2000,FMIN_CIV_2000,AI_CIV,ERR_AI_CIV,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,POSMIN_CIV_450,FMIN_CIV_450,BI_SIIV,ERR_BI_SIIV,NSIIV_2000,VMIN_SIIV_2000,VMAX_SIIV_2000,POSMIN_SIIV_2000,FMIN_SIIV_2000,AI_SIIV,ERR_AI_SIIV,NSIIV_450,VMIN_SIIV_450,VMAX_SIIV_450,POSMIN_SIIV_450,FMIN_SIIV_450,BALMASK,Z_MASK,ZERR_MASK,ZWARN_MASK,CHI2_MASK,DELTACHI2_MASK,SPECTYPE_MASK,SNR_CIV
,,,,,,deg,deg,,mag,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,,,,,,,,
int64,float64,float64,int64,str6,int32,float64,float64,str4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,int16,int64,int64,int64,int16,float32,int64,int64,int64,int64,int64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int64,str3,str4,float32[5],float32,float32,float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],uint8,float64,float64,int64,float64,float64,str6,float64
1083386135314440,3.631155815059421,0.0007740990556025613,4,QSO,0,150.1172290943953,3.203639817689895,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.997675796508789,0,0,0,0,4,5545.4507,0,0,4611686018427387904,0,70368744177664,0,205.45746,108.59245,--,--,--,--,--,--,--,3.6311557,1.5085001,0.0007476383,4.9172184e-05,0.0020246075,0.00030511015,0.00014915048,0.00031868834,3.9982815,4.1089396,1.5085001,0.32323426,0.33692527,0.22890207,2,27259,sv3,dark,0.38701344 .. -0.13898505,1.0797844,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0,-1.0,-1.0,-1,-1.0,-1.0,-1,0.49071757215759687
1083393307574272,2.4328130904523575,0.0015506419586017728,4,GALAXY,0,218.11601975479252,3.456844830548416,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.997675796508789,0,0,0,0,1,1023.7778,0,0,4611686018427387904,0,70368744177664,0,33.64711,31.710672,3.9321053,0.11392317,16.187904,-0.008994076,0.0032281901,90.45671,0.00033598195,1.225154,2.4456637,0.011825093,0.018332023,0.6120433,0.98057455,2.510112e-05,1.2357961e-05,2.437483,2.504086,2.451604,2.4456637,0.48333326,0.46334296,24,25979,sv3,dark,3.6173832 .. -0.3486411,0.9208186,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0,-1.0,-1.0,-1,-1.0,-1.0,-1,0.5238561644399842
1084093743759365,3.6455239

In [ ]:
cor_cat = Table.read(fits.open('/home/colgoat/spc_DESIedr/zall-pix-fuji.fits')[1])
cor_cat[(cor_cat['PROGRAM']=='dark') & ((cor_cat['SURVEY']=='sv1') | (cor_cat['SURVEY']=='sv3'))]